In [1]:
import akshare as ak
# import common_mysql as common
# import common_pg as common
import sys
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common')
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py')

print("sys.path:",sys.path)

# import common.common_pg as common_db

import common_pg as common_db

print(ak.__version__)

sys.path: ['/Users/afirez/studio/python/auto-trade/examples/py_quant/datasource/akshare/test', '/usr/local/anaconda3/envs/py38/lib/python38.zip', '/usr/local/anaconda3/envs/py38/lib/python3.8', '/usr/local/anaconda3/envs/py38/lib/python3.8/lib-dynload', '', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions', '/Users/afirez/.ipython', '/Users/afirez/.local/lib/python3.8/site-packages', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py']
pg_host : 192.168.31.110 ,pg_user : postgres ,pg_db : k_house
1.6.64


### 历史行情数据-新浪

接口: stock_zh_a_daily

P.S. 建议切换为 stock_zh_a_hist 接口使用(该接口数据质量高, 访问无限制)

目标地址: https://finance.sina.com.cn/realstock/company/sh600006/nc.shtml(示例)

描述: 新浪财经-沪深京 A 股的数据, 历史数据按日频率更新; 注意其中的 sh689009 为 CDR, 请 通过 ak.stock_zh_a_cdr_daily 接口获取

限量: 单次返回指定沪深京 A 股上市公司指定日期间的历史行情日频率数据, 多次获取容易封禁 IP

In [6]:
symbol="002594"

stock_zh_a_daily_qfq_df = ak.stock_zh_a_daily(symbol=symbol, adjust="qfq")
print(stock_zh_a_daily_qfq_df)

           date    open    high     low   close      volume  \
0    2011-06-30   21.55   25.66   21.55   24.93  56292536.0   
1    2011-07-01   25.25   27.43   24.49   27.43  33561280.0   
2    2011-07-04   28.29   30.17   28.00   30.17  21964592.0   
3    2011-07-05   32.11   33.19   31.24   32.38  38108084.0   
4    2011-07-06   32.13   34.83   31.46   32.72  31296720.0   
...         ...     ...     ...     ...     ...         ...   
2678 2022-07-25  325.89  326.89  319.90  322.85  11501710.0   
2679 2022-07-26  322.38  327.57  319.90  324.90  10765616.0   
2680 2022-07-27  323.90  329.69  320.91  325.69   9105572.0   
2681 2022-07-28  328.91  331.34  323.90  325.33  10906517.0   
2682 2022-07-29  323.00  327.80  317.96  320.36  11485364.0   

      outstanding_share  turnover  
0          6.400000e+07  0.879571  
1          6.400000e+07  0.524395  
2          6.400000e+07  0.343197  
3          6.400000e+07  0.595439  
4          6.400000e+07  0.489011  
...                 ...    

In [ ]:
# common_db.insert_db(stock_zh_a_daily_qfq_df, "stock_zh_a_daily", True, "`symbol`")

In [7]:
# 0. 准备工作

dbname = "k_house"
#   0.1 创建数据库连接

#  创建数据库 股票分析数据库 的连接 
# conn, curs, engine = common_db.creat_conn_with_pg(dbname=f'{dbname}_for_test')

#  创建数据库 k_house 的连接 
conn2, curs2, engine2 = common_db.creat_conn_with_pg(dbname=dbname)

In [8]:
import pandas as pd

In [9]:
table_name = 'stock_zh_a_daily_test'

stock_zh_a_daily_qfq_df.to_sql(table_name,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

symbol

'sz002594'

In [10]:
df = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name}
  """,
   # params = (date,),
  con = engine2,
)

df.head()

,date,open,high,low,close,volume,outstanding_share,turnover
0,2011-06-30,21.55,25.66,21.55,24.93,56292536.0,64000000.0,0.879571
1,2011-07-01,25.25,27.43,24.49,27.43,33561280.0,64000000.0,0.524395
2,2011-07-04,28.29,30.17,28.00,30.17,21964592.0,64000000.0,0.343197
3,2011-07-05,32.11,33.19,31.24,32.38,38108084.0,64000000.0,0.595439
4,2011-07-06,32.13,34.83,31.46,32.72,31296720.0,64000000.0,0.489011


In [11]:
df[:-5]

,date,open,high,low,close,volume,outstanding_share,turnover
0,2011-06-30,21.55,25.66,21.55,24.93,56292536.0,6.400000e+07,0.879571
1,2011-07-01,25.25,27.43,24.49,27.43,33561280.0,6.400000e+07,0.524395
2,2011-07-04,28.29,30.17,28.00,30.17,21964592.0,6.400000e+07,0.343197
3,2011-07-05,32.11,33.19,31.24,32.38,38108084.0,6.400000e+07,0.595439
4,2011-07-06,32.13,34.83,31.46,32.72,31296720.0,6.400000e+07,0.489011
...,...,...,...,...,...,...,...,...
2673,2022-07-18,344.89,345.89,327.89,336.99,23891156.0,1.164765e+09,0.020512
2674,2022-07-19,336.99,336.99,327.51,330.99,15818025.0,1.164765e+09,0.013580
2675,2022-07-20,332.87,333.75,328.97,331.59,10326348.0,1.164765e+09,0.008866
2676,2022-07-21,328.89,330.89,322.84,324.40,13200328.0,1.164765e+09,0.011333


### 历史行情数据-东财

接口: stock_zh_a_hist

目标地址: http://quote.eastmoney.com/concept/sh603777.html?from=classic(示例)

描述: 东方财富-沪深京 A 股日频率数据; 历史数据按日频率更新, 当日收盘价请在收盘后获取

限量: 单次返回指定沪深京 A 股上市公司、指定周期和指定日期间的历史行情日频率数据

In [13]:
symbol="002594"

In [14]:
stock_zh_a_hist_df = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date="20000301", end_date='20220729', adjust="")
print(stock_zh_a_hist_df)

              日期      开盘      收盘      最高      最低     成交量           成交额     振幅  \
0     2011-06-30   22.00   25.45   26.19   22.00  562925  1.349342e+09  23.28   
1     2011-07-01   25.77   28.00   28.00   25.00  335613  8.986792e+08  11.79   
2     2011-07-04   28.88   30.80   30.80   28.58  219646  6.535179e+08   7.93   
3     2011-07-05   32.78   33.05   33.88   31.89  381081  1.257046e+09   6.46   
4     2011-07-06   32.80   33.40   35.55   32.11  312967  1.056614e+09  10.41   
...          ...     ...     ...     ...     ...     ...           ...    ...   
2678  2022-07-25  326.00  322.95  327.00  320.00  115017  3.705167e+09   2.12   
2679  2022-07-26  322.48  325.00  327.68  320.00  107656  3.493834e+09   2.38   
2680  2022-07-27  324.00  325.80  329.80  321.01   91056  2.961812e+09   2.70   
2681  2022-07-28  329.02  325.43  331.45  324.00  109065  3.572559e+09   2.29   
2682  2022-07-29  323.00  320.36  327.80  317.96  114854  3.695132e+09   3.02   

        涨跌幅   涨跌额    换手率  


In [15]:
table_name2 = 'stock_zh_a_hist_daily_test'

stock_zh_a_hist_df.to_sql(table_name2,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [16]:
df2 = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name2}
  """,
   # params = (date,),
  con = engine2,
)

df2[:-5]

,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,2011-06-30,22.00,25.45,26.19,22.00,562925,1.349342e+09,23.28,41.39,7.45,87.96
1,2011-07-01,25.77,28.00,28.00,25.00,335613,8.986792e+08,11.79,10.02,2.55,52.44
2,2011-07-04,28.88,30.80,30.80,28.58,219646,6.535179e+08,7.93,10.00,2.80,34.32
3,2011-07-05,32.78,33.05,33.88,31.89,381081,1.257046e+09,6.46,7.31,2.25,59.54
4,2011-07-06,32.80,33.40,35.55,32.11,312967,1.056614e+09,10.41,1.06,0.35,48.90
...,...,...,...,...,...,...,...,...,...,...,...
2673,2022-07-18,345.00,337.10,346.00,328.00,238912,8.007560e+09,5.32,-0.32,-1.09,2.05
2674,2022-07-19,337.10,331.10,337.10,327.62,158180,5.239778e+09,2.81,-1.78,-6.00,1.36
2675,2022-07-20,332.98,331.70,333.86,329.08,103263,3.425067e+09,1.44,0.18,0.60,0.89
2676,2022-07-21,329.00,324.50,331.00,322.94,132003,4.303074e+09,2.43,-2.17,-7.20,1.13


### 历史行情数据-网易

接口: stock_zh_a_hist_163

目标地址: http://quotes.money.163.com/trade/lsjysj_601318.html?year=2022&season=2

描述: 网易财经-行情首页-沪深 A 股-每日行情; 该接口主要用户获取流通市值、总市值等指标

限量: 单次返回指定沪深 A 股（不包含北交所）上市公司指定日期间的历史行情日频率数据, 该接口只返回未复权数据

In [23]:
stock_zh_a_hist_163_df = ak.stock_zh_a_hist_163(symbol="sz002594", start_date="20000101", end_date="20220729")
print(stock_zh_a_hist_163_df)

              日期    股票代码    名称     收盘价     最高价     最低价     开盘价     前收盘   涨跌额  \
0     2011-06-21  002594   比亚迪    0.00    0.00    0.00    0.00   18.00   NaN   
1     2011-06-30  002594  N比亚迪   25.45   26.19   22.00   22.00   18.00  7.45   
2     2011-07-01  002594   比亚迪   28.00   28.00   25.00   25.77   25.45  2.55   
3     2011-07-04  002594   比亚迪   30.80   30.80   28.58   28.88   28.00  2.80   
4     2011-07-05  002594   比亚迪   33.05   33.88   31.89   32.78   30.80  2.25   
...          ...     ...   ...     ...     ...     ...     ...     ...   ...   
2691  2022-07-25  002594   比亚迪  322.95  327.00  320.00  326.00  329.62 -6.67   
2692  2022-07-26  002594   比亚迪  325.00  327.68  320.00  322.48  322.95  2.05   
2693  2022-07-27  002594   比亚迪  325.80  329.80  321.01  324.00  325.00  0.80   
2694  2022-07-28  002594   比亚迪  325.43  331.45  324.00  329.02  325.80 -0.37   
2695  2022-07-29  002594   比亚迪  320.36  327.80  317.96  323.00  325.33 -4.97   

          涨跌幅      换手率         成交量     

In [24]:
table_name_163 = 'stock_zh_a_hist_163_daily_test'

stock_zh_a_hist_163_df.to_sql(table_name_163,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [25]:
df_163 = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_163}
  """,
   # params = (date,),
  con = engine2,
)

df_163[:-5]

,日期,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,换手率,成交量,成交金额,总市值,流通市值
0,2011-06-21,002594,比亚迪,0.00,0.00,0.00,0.00,18.00,NaN,NaN,0.0000,0.0,0.000000e+00,4.095180e+10,0.000000e+00
1,2011-06-30,002594,N比亚迪,25.45,26.19,22.00,22.00,18.00,7.45,41.3889,87.9571,56292534.0,1.349342e+09,5.991184e+10,1.628800e+09
2,2011-07-01,002594,比亚迪,28.00,28.00,25.00,25.77,25.45,2.55,10.0196,52.4395,33561279.0,8.986792e+08,6.591480e+10,1.792000e+09
3,2011-07-04,002594,比亚迪,30.80,30.80,28.58,28.88,28.00,2.80,10.0000,34.3197,21964591.0,6.535179e+08,7.250628e+10,1.971200e+09
4,2011-07-05,002594,比亚迪,33.05,33.88,31.89,32.78,30.80,2.25,7.3052,59.5439,38108083.0,1.257046e+09,7.780300e+10,2.115200e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2686,2022-07-18,002594,比亚迪,337.10,346.00,328.00,345.00,338.19,-1.09,-0.3223,1.3177,23891156.0,8.007560e+09,9.813463e+11,6.112105e+11
2687,2022-07-19,002594,比亚迪,331.10,337.10,327.62,337.10,337.10,-6.00,-1.7799,0.8724,15818025.0,5.239778e+09,9.638794e+11,6.003316e+11
2688,2022-07-20,002594,比亚迪,331.70,333.86,329.08,332.98,331.10,0.60,0.1812,0.5695,10326348.0,3.425067e+09,9.656261e+11,6.014195e+11
2689,2022-07-21,002594,比亚迪,324.50,331.00,322.94,329.00,331.70,-7.20,-2.1706,0.7280,13200328.0,4.303074e+09,9.446659e+11,5.883649e+11


### 分时数据-新浪
接口: stock_zh_a_minute

目标地址: http://finance.sina.com.cn/realstock/company/sh600519/nc.shtml

描述: 新浪财经-沪深京 A 股股票或者指数的分时数据，目前可以获取 1, 5, 15, 30, 60 分钟的数据频率, 可以指定是否复权

限量: 单次返回指定股票或指数的指定频率的最近交易日的历史分时行情数据

In [32]:
stock_zh_a_minute_df = ak.stock_zh_a_minute(symbol='sh600751', period='1', adjust="qfq")
print(stock_zh_a_minute_df)

                       day  open  high   low  close  volume
0      2021-12-09 11:02:00  2.76  2.76  2.75   2.75   44100
1      2021-12-09 11:03:00  2.76  2.76  2.75   2.76    2500
2      2021-12-09 11:04:00  2.75  2.76  2.75   2.76    3300
3      2021-12-09 11:05:00  2.75  2.76  2.75   2.76    4600
4      2021-12-09 11:06:00  2.76  2.76  2.75   2.75    7000
...                    ...   ...   ...   ...    ...     ...
36575  2022-07-29 14:54:00  2.60  2.61  2.60   2.61   72000
36576  2022-07-29 14:55:00  2.60  2.61  2.60   2.61   68800
36577  2022-07-29 14:56:00  2.61  2.61  2.61   2.61   71600
36578  2022-07-29 14:57:00  2.61  2.62  2.61   2.61   67100
36579  2022-07-29 15:00:00  2.60  2.60  2.60   2.60  229100

[36580 rows x 6 columns]


In [33]:
table_name_sina_2 = 'stock_zh_a_hist_minute_sina_test'

stock_zh_a_minute_df.to_sql(table_name_sina_2,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [34]:
df_sina_2= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_sina_2}
  """,
   # params = (date,),
  con = engine2,
)

df_sina_2[:-5]

,day,open,high,low,close,volume
0,2021-12-09 11:02:00,2.76,2.76,2.75,2.75,44100
1,2021-12-09 11:03:00,2.76,2.76,2.75,2.76,2500
2,2021-12-09 11:04:00,2.75,2.76,2.75,2.76,3300
3,2021-12-09 11:05:00,2.75,2.76,2.75,2.76,4600
4,2021-12-09 11:06:00,2.76,2.76,2.75,2.75,7000
...,...,...,...,...,...,...
36570,2022-07-29 14:49:00,2.61,2.61,2.61,2.61,37400
36571,2022-07-29 14:50:00,2.60,2.62,2.60,2.61,281000
36572,2022-07-29 14:51:00,2.61,2.61,2.61,2.61,18700
36573,2022-07-29 14:52:00,2.60,2.61,2.60,2.61,26700


### 分时数据-东财

接口: stock_zh_a_hist_min_em

目标地址: http://quote.eastmoney.com/concept/sh603777.html?from=classic

描述: 东方财富网-行情首页-沪深京 A 股-每日分时行情; 该接口只能获取近期的分时数据，注意时间周期的设置

限量: 单次返回指定股票、频率、复权调整和时间区间的分时数据, 其中 1 分钟数据只返回近 5 个交易日数据且不复权

In [37]:
import akshare as ak

stock_zh_a_hist_min_em_df = ak.stock_zh_a_hist_min_em(symbol="002594", start_date="2000-09-01 09:32:00", end_date="2022-07-29 09:32:00", period='1', adjust='')
print(stock_zh_a_hist_min_em_df)

                      时间      开盘      收盘      最高      最低   成交量          成交额  \
0    2022-07-25 09:30:00    0.00  326.00  326.00  326.00  1057   34458200.0   
1    2022-07-25 09:31:00    0.00  323.97  327.00  323.00  3393  110074997.0   
2    2022-07-25 09:32:00    0.00  323.66  324.55  323.42  2022   65496639.0   
3    2022-07-25 09:33:00    0.00  323.01  323.65  322.02  2000   64554897.0   
4    2022-07-25 09:34:00    0.00  323.66  323.67  322.50  1543   49823201.0   
..                   ...     ...     ...     ...     ...   ...          ...   
962  2022-07-28 14:59:00    0.00  325.42  325.42  325.42     0          0.0   
963  2022-07-28 15:00:00    0.00  325.43  325.43  325.43  1738   56575029.0   
964  2022-07-29 09:30:00  323.00  323.00  323.00  323.00   523   16892900.0   
965  2022-07-29 09:31:00  323.90  324.62  325.30  322.88  1362   44179143.0   
966  2022-07-29 09:32:00  324.48  325.57  326.35  324.48  1268   41269857.0   

         最新价  
0    326.000  
1    324.794  
2    3

In [38]:
table_name_em_2 = 'stock_zh_a_hist_minute_em_test'

stock_zh_a_minute_df.to_sql(table_name_em_2,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [39]:
df_em_2= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_em_2}
  """,
   # params = (date,),
  con = engine2,
)

df_em_2[:-5]

,day,open,high,low,close,volume
0,2021-12-09 11:02:00,2.76,2.76,2.75,2.75,44100
1,2021-12-09 11:03:00,2.76,2.76,2.75,2.76,2500
2,2021-12-09 11:04:00,2.75,2.76,2.75,2.76,3300
3,2021-12-09 11:05:00,2.75,2.76,2.75,2.76,4600
4,2021-12-09 11:06:00,2.76,2.76,2.75,2.75,7000
...,...,...,...,...,...,...
36570,2022-07-29 14:49:00,2.61,2.61,2.61,2.61,37400
36571,2022-07-29 14:50:00,2.60,2.62,2.60,2.61,281000
36572,2022-07-29 14:51:00,2.61,2.61,2.61,2.61,18700
36573,2022-07-29 14:52:00,2.60,2.61,2.60,2.61,26700


#### 盘前数据

接口: stock_zh_a_hist_pre_min_em

目标地址: http://quote.eastmoney.com/concept/sh603777.html?from=classic

描述: 东方财富-股票行情-盘前数据

限量: 单次返回指定当前交易日的股票分钟数据, 包含盘前分钟数据

In [40]:
stock_zh_a_hist_pre_min_em_df = ak.stock_zh_a_hist_pre_min_em(symbol="002594")
print(stock_zh_a_hist_pre_min_em_df)

                      时间      开盘      收盘      最高      最低   成交量         成交额  \
0    2022-07-29 09:15:00  325.33  325.33  325.33  325.33     0         0.0   
1    2022-07-29 09:16:00  327.00  325.33  327.00  325.33     0         0.0   
2    2022-07-29 09:17:00  325.33  325.33  325.33  325.33     0         0.0   
3    2022-07-29 09:18:00  325.33  325.33  325.33  325.33     0         0.0   
4    2022-07-29 09:19:00  325.33  325.33  325.33  325.33     0         0.0   
..                   ...     ...     ...     ...     ...   ...         ...   
251  2022-07-29 14:56:00  320.18  320.14  320.21  320.14   685  21940311.0   
252  2022-07-29 14:57:00  320.15  320.27  320.29  320.15   525  16822422.0   
253  2022-07-29 14:58:00  320.27  320.27  320.27  320.27    16    512439.0   
254  2022-07-29 14:59:00  320.27  320.27  320.27  320.27     0         0.0   
255  2022-07-29 15:00:00  320.36  320.36  320.36  320.36  1657  53083652.0   

         最新价  
0    325.330  
1    325.330  
2    325.330  
3  

In [41]:
table_name_em_3 = 'stock_zh_a_hist_pre_minute_em_test'

stock_zh_a_hist_pre_min_em_df.to_sql(table_name_em_3,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [42]:
df_em_3= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_em_3}
  """,
   # params = (date,),
  con = engine2,
)

df_em_3[:-5]

,时间,开盘,收盘,最高,最低,成交量,成交额,最新价
0,2022-07-29 09:15:00,325.33,325.33,325.33,325.33,0,0.0,325.330
1,2022-07-29 09:16:00,327.00,325.33,327.00,325.33,0,0.0,325.330
2,2022-07-29 09:17:00,325.33,325.33,325.33,325.33,0,0.0,325.330
3,2022-07-29 09:18:00,325.33,325.33,325.33,325.33,0,0.0,325.330
4,2022-07-29 09:19:00,325.33,325.33,325.33,325.33,0,0.0,325.330
...,...,...,...,...,...,...,...,...
246,2022-07-29 14:51:00,320.20,320.19,320.20,320.17,540,17290357.0,321.788
247,2022-07-29 14:52:00,320.19,320.18,320.19,320.14,379,12134605.0,321.782
248,2022-07-29 14:53:00,320.18,320.18,320.18,320.15,470,15047703.0,321.775
249,2022-07-29 14:54:00,320.18,320.20,320.20,320.15,485,15528808.0,321.768


### 历史分笔数据-腾讯财经

接口: stock_zh_a_tick_tx

目标地址: http://gu.qq.com/sz300494/gp/detail(示例)

描述: A 股数据是从腾讯财经获取的数据, 历史数据按日频率更新, 该接口自 20210611 起, 暂时无法获取数据, 等腾讯修复后再使用

限量: 单次返回具体某个 A 上市公司的近 2 年历史分笔行情数据

P.S. 每个交易日 16:00 提供当日数据; 如遇到数据缺失, 请使用 ak.stock_zh_a_tick_163() 接口(注意数据会有一定差异)或者使用 ak.stock_zh_a_tick_tx_js(symbol=”sz000001”) 来获取当日腾讯分笔数据

In [55]:
stock_zh_a_tick_tx_df = ak.stock_zh_a_tick_tx(symbol="sh600848", trade_date="20191011")
print(stock_zh_a_tick_tx_df)


          成交时间   成交价格  价格变动  成交量(手)   成交额(元)   性质
0     09:25:04  26.17  0.18     163   426571   买盘
1     09:30:03  26.17  0.00     163   426636   卖盘
2     09:30:06  26.14 -0.03      38    99386  中性盘
3     09:30:11  26.15  0.01     110   287650   卖盘
4     09:30:15  26.14 -0.01       5    13078   卖盘
...        ...    ...   ...     ...      ...  ...
1978  14:56:43  26.19 -0.01      30    78584   卖盘
1979  14:56:46  26.19  0.00       1     2619   买盘
1980  14:56:52  26.21  0.02      26    68097   买盘
1981  14:56:54  26.18 -0.03       7    18326   卖盘
1982  15:00:02  26.19  0.01     591  1547829   卖盘

[1983 rows x 6 columns]


In [56]:
table_name_tx_2 = 'stock_zh_a_tick_tx_test'

stock_zh_a_tick_tx_df.to_sql(table_name_tx_2,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [57]:
df_tx_2= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_tx_2}
  """,
   # params = (date,),
  con = engine2,
)

df_tx_2[:-5]

,成交时间,成交价格,价格变动,成交量(手),成交额(元),性质
0,09:25:04,26.17,0.18,163,426571,买盘
1,09:30:03,26.17,0.00,163,426636,卖盘
2,09:30:06,26.14,-0.03,38,99386,中性盘
3,09:30:11,26.15,0.01,110,287650,卖盘
4,09:30:15,26.14,-0.01,5,13078,卖盘
...,...,...,...,...,...,...
1973,14:56:18,26.19,0.01,81,212139,买盘
1974,14:56:31,26.20,0.01,10,26200,买盘
1975,14:56:33,26.20,0.00,5,13100,买盘
1976,14:56:37,26.19,-0.01,25,65495,卖盘


In [58]:
# 当前交易日的分时数据可以通过下接口获取
stock_zh_a_tick_tx_js_df = ak.stock_zh_a_tick_tx_js(symbol="sz000001")
print(stock_zh_a_tick_tx_js_df)

/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages/akshare/stock/stock_zh_a_tick_tx_163.py:28: UserWarning: 正在下载数据，请稍等
  warnings.warn("正在下载数据，请稍等")


          成交时间   成交价格  价格变动    成交量      成交金额  性质
0     09:25:00  12.88  0.00   1491   1920408  卖盘
1     09:30:00  12.88  0.00    503    648138  卖盘
2     09:30:03  12.88  0.00   2114   2723744  卖盘
3     09:30:06  12.87 -0.01   1073   1381553  卖盘
4     09:30:09  12.88  0.01    856   1101793  买盘
...        ...    ...   ...    ...       ...  ..
4634  14:56:51  12.68 -0.01    775    982703  卖盘
4635  14:56:54  12.67 -0.01    165    209128  卖盘
4636  14:56:57  12.68  0.01    303    384213  买盘
4637  14:57:00  12.68  0.00     59     74807  买盘
4638  15:00:03  12.68  0.00  15548  19714864  卖盘

[4639 rows x 6 columns]


/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages/akshare/stock/stock_zh_a_tick_tx_163.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_df["性质"] = big_df["性质"].map(property_map)


In [59]:
table_name_tx_3 = 'stock_zh_a_tick_tx_js_test'

stock_zh_a_tick_tx_js_df.to_sql(table_name_tx_3,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [60]:
df_tx_3= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_tx_3}
  """,
   # params = (date,),
  con = engine2,
)

df_tx_3[:-5]

,成交时间,成交价格,价格变动,成交量,成交金额,性质
0,09:25:00,12.88,0.00,1491,1920408,卖盘
1,09:30:00,12.88,0.00,503,648138,卖盘
2,09:30:03,12.88,0.00,2114,2723744,卖盘
3,09:30:06,12.87,-0.01,1073,1381553,卖盘
4,09:30:09,12.88,0.01,856,1101793,买盘
...,...,...,...,...,...,...
4629,14:56:36,12.67,-0.01,1158,1467230,卖盘
4630,14:56:39,12.68,0.01,225,285156,买盘
4631,14:56:42,12.68,0.00,807,1023264,买盘
4632,14:56:45,12.68,0.00,81,102747,卖盘


### 分笔数据-网易财经

接口: stock_zh_a_tick_163

目标地址: http://quotes.money.163.com/trade/cjmx_000001.html#01b05(示例)

描述: A 股 Tick 数据是从网易财经获取, 历史数据按日频率更新, 晚上 10 更新数据; 可以调用 ak.stock_zh_a_tick_163_now(symbol=”000001”) 接口获取 当日数据(该接口只能在交易日获取数据), 主要参数无市场标识

限量: 单次返回具体某个 A 上市公司的近 5 个交易日的历史分笔行情数据; 该接口目前还不支持北交所的股票; 该接口目前不能返回 上交所 数据

In [63]:
stock_zh_a_tick_163_df = ak.stock_zh_a_tick_163(symbol="sz002594", trade_date="20220729")
print(stock_zh_a_tick_163_df)

            时间     成交价  价格变动   成交量         成交额   性质
0     09:30:03  323.90  0.90   125   4046646.0   买盘
1     09:30:05  323.99  0.09   215   6957273.0   买盘
2     09:30:11  323.99  0.00    49   1589851.0   买盘
3     09:30:15  324.00  0.01    36   1168107.0  中性盘
4     09:30:19  324.62  0.62    14    454506.0   卖盘
...        ...     ...   ...   ...         ...  ...
4320  14:56:55  320.27 -0.02    17    544450.0   卖盘
4321  14:56:58  320.28  0.01    10    320276.0   买盘
4322  14:57:00  320.27 -0.01    21    672574.0   卖盘
4323  14:57:03  320.27  0.00    16    512439.0   卖盘
4324  15:00:03  320.36  0.09  1657  53083652.0   卖盘

[4325 rows x 6 columns]


In [64]:
table_name_163_4 = 'stock_zh_a_tick_163_test'

stock_zh_a_tick_163_df.to_sql(table_name_163_4,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [65]:
df_163_4= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_163_4}
  """,
   # params = (date,),
  con = engine2,
)

df_163_4[:-5]

,时间,成交价,价格变动,成交量,成交额,性质
0,09:30:03,323.90,0.90,125,4046646.0,买盘
1,09:30:05,323.99,0.09,215,6957273.0,买盘
2,09:30:11,323.99,0.00,49,1589851.0,买盘
3,09:30:15,324.00,0.01,36,1168107.0,中性盘
4,09:30:19,324.62,0.62,14,454506.0,卖盘
...,...,...,...,...,...,...
4315,14:56:39,320.24,0.00,47,1505110.0,中性盘
4316,14:56:42,320.26,0.02,34,1088883.0,中性盘
4317,14:56:45,320.26,0.00,13,416347.0,卖盘
4318,14:56:48,320.27,0.01,43,1377192.0,中性盘


## 股票信息

 "stock_info_sz_name_code"  # 深证证券交易所股票代码和简称

 "stock_info_sh_name_code"  # 上海证券交易所股票代码和简称

 "stock_info_bj_name_code"  # 北京证券交易所股票代码和简称

 "stock_info_sh_delist"  # 上海证券交易所暂停和终止上市

 "stock_info_sz_delist"  # 深证证券交易所暂停和终止上市

 "stock_info_sz_change_name"  # 深证证券交易所股票曾用名详情

 "stock_info_change_name"  # A 股股票曾用名列表

 "stock_info_a_code_name"  # A 股股票代码和简称

In [66]:
stock_info_a_code_name_df = ak.stock_info_a_code_name()
print(stock_info_a_code_name_df)

100%|██████████| 6/6 [00:01<00:00,  5.04it/s]

        code   name
0     000001   平安银行
1     000002  万  科Ａ
2     000004   ST国华
3     000005   ST星源
4     000006   深振业Ａ
...      ...    ...
4839  871970   大禹生物
4840  871981   晶赛科技
4841  872925   锦好医疗
4842  873169   七丰精工
4843  873223   荣亿精密

[4844 rows x 2 columns]


In [73]:
table_name_stock_info_a_code_name = 'stock_info_a_code_name'

stock_info_a_code_name_df.to_sql(table_name_stock_info_a_code_name,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute(
#     """
#     CREATE  INDEX stock_info_a_code_name_code  ON  stock_info_a_code_name  (code); 
#     """
# )
# conn2.commit()

In [74]:
df_stock_info_a_code_name= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_stock_info_a_code_name}
  """,
   # params = (date,),
  con = engine2,
)

df_stock_info_a_code_name[:-5]

,code,name
0,000001,平安银行
1,000002,万 科Ａ
2,000004,ST国华
3,000005,ST星源
4,000006,深振业Ａ
...,...,...
4834,871245,威博液压
4835,871396,常辅股份
4836,871553,凯腾精工
4837,871642,通易航天


 ## 股票板块

 "stock_sector_spot"  # 板块行情

 "stock_sector_detail"  # 板块详情(具体股票)

In [75]:
stock_sector_spot_df = ak.stock_sector_spot()
print(stock_sector_spot_df)

        label    板块  公司家数        平均价格       涨跌额       涨跌幅        总成交量  \
0    new_blhy  玻璃行业    19   17.157895  0.303158  1.798651   555396832   
1    new_cbzz  船舶制造     8   11.281250 -0.240000 -2.083107   257759237   
2    new_cmyl  传媒娱乐    40    6.597838 -0.037027 -0.558068   479650890   
3    new_dlhy  电力行业    62    7.605167  0.041000  0.542029  2320694276   
4    new_dqhy  电器行业    58   16.133158  0.084386  0.525809  2047879519   
5    new_dzqj  电子器件   152   19.755743 -0.441351 -2.185222  4296841326   
6    new_dzxx  电子信息   247   11.809530 -0.061667 -0.519465  5263561167   
7     new_fdc   房地产   123    7.120342 -0.133333 -1.838149  3533336572   
8    new_fdsb  发电设备    65   11.148387  0.084516  0.763893  2837956617   
9    new_fjzz  飞机制造    14   21.612143 -0.550000 -2.481709   250061579   
10   new_fzhy  纺织行业    42    6.483333 -0.022564 -0.346825   595249283   
11   new_fzjx  纺织机械     8   10.765000 -0.076250 -0.703332    66031237   
12   new_fzxl  服装鞋类    49    8.859167 -0.041667 -0.

In [77]:
table_name_stock_sector_spot_sina = 'stock_sector_spot_sina'

stock_sector_spot_df.to_sql(table_name_stock_sector_spot_sina,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute(
#     """
#     CREATE  INDEX stock_info_a_code_name_code  ON  stock_info_a_code_name  (code); 
#     """
# )
# conn2.commit()

In [78]:
df_stock_sector_spot_sina_2= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_stock_sector_spot_sina}
  """,
   # params = (date,),
  con = engine2,
)

df_stock_sector_spot_sina_2[:-5]

,label,板块,公司家数,平均价格,涨跌额,涨跌幅,总成交量,总成交额,股票代码,个股-涨跌幅,个股-当前价,个股-涨跌额,股票名称
0,new_blhy,玻璃行业,19,17.157895,0.303158,1.798651,555396832,6432665426,sh600876,10.013,25.71,2.34,洛阳玻璃
1,new_cbzz,船舶制造,8,11.281250,-0.240000,-2.083107,257759237,3031560866,sz002608,2.256,6.80,0.15,江苏国信
2,new_cmyl,传媒娱乐,40,6.597838,-0.037027,-0.558068,479650890,3402536617,sh600551,7.962,11.39,0.84,时代出版
3,new_dlhy,电力行业,62,7.605167,0.041000,0.542029,2320694276,15924750466,sz000531,9.974,8.60,0.78,穗恒运Ａ
4,new_dqhy,电器行业,58,16.133158,0.084386,0.525809,2047879519,26402140945,sz002665,10.109,4.03,0.37,首航高科
5,new_dzqj,电子器件,152,19.755743,-0.441351,-2.185222,4296841326,63849133403,sh600355,10.030,3.73,0.34,精伦电子
6,new_dzxx,电子信息,247,11.809530,-0.061667,-0.519465,5263561167,54943314987,sz300044,19.943,4.21,0.70,赛为智能
7,new_fdc,房地产,123,7.120342,-0.133333,-1.838149,3533336572,22313824519,sh600615,4.461,8.43,0.36,*ST丰华
8,new_fdsb,发电设备,65,11.148387,0.084516,0.763893,2837956617,28457656328,sz300062,19.973,8.83,1.47,中能电气
9,new_fjzz,飞机制造,14,21.612143,-0.550000,-2.481709,250061579,5887615337,sh600118,1.041,22.32,0.23,中国卫星


 ## 机构持股

 "stock_institute_hold"  # 机构持股一览表
 "stock_institute_hold_detail"  # 机构持股详情

In [79]:
stock_institute_hold_sina_df = ak.stock_institute_hold()
print(stock_institute_hold_sina_df)

       证券代码  证券简称  机构数  机构数变化   持股比例  持股比例增幅  占流通股比例  占流通股比例增幅
0    000776  广发证券    1      0   1.85   -1.44    3.66     -2.85
1    000619  海螺型材    1     -1   0.76    0.02    1.69      0.04
2    600281  ST太化    1      0   0.11    0.00    0.36      0.00
3    000410  沈阳机床    1     -1   0.11   -0.12    0.29     -0.29
4    600507  方大特钢    1      0   0.92   -0.26    2.31     -0.65
..      ...   ...  ...    ...    ...     ...     ...       ...
600  002024  苏宁电器   10      0   9.70   -0.74   36.14     -2.77
601  600825  新华传媒   10      0  17.55    0.08   37.24      0.17
602  600019  宝钢股份   10      0   3.12    0.17   20.78      1.11
603  600085   同仁堂   10      0  12.49    0.59   34.89      1.66
604  002023  海特高新   10      0  14.15    2.03   46.22      6.62

[605 rows x 8 columns]


In [80]:
table_name_stock_institute_hold_sina = 'stock_institute_hold_sina'

stock_institute_hold_sina_df.to_sql(table_name_stock_institute_hold_sina,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute(
#     """
#     CREATE  INDEX stock_info_a_code_name_code  ON  stock_info_a_code_name  (code); 
#     """
# )
# conn2.commit()

In [81]:
df_stock_institute_hold_sina_2= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_stock_institute_hold_sina}
  """,
   # params = (date,),
  con = engine2,
)

df_stock_institute_hold_sina_2[:-5]

,证券代码,证券简称,机构数,机构数变化,持股比例,持股比例增幅,占流通股比例,占流通股比例增幅
0,000776,广发证券,1,0,1.85,-1.44,3.66,-2.85
1,000619,海螺型材,1,-1,0.76,0.02,1.69,0.04
2,600281,ST太化,1,0,0.11,0.00,0.36,0.00
3,000410,沈阳机床,1,-1,0.11,-0.12,0.29,-0.29
4,600507,方大特钢,1,0,0.92,-0.26,2.31,-0.65
...,...,...,...,...,...,...,...,...
595,600089,特变电工,10,0,10.25,-1.82,18.05,-3.21
596,600125,铁龙物流,10,0,14.13,1.48,26.46,2.76
597,600267,海正药业,10,0,10.68,-1.73,41.66,-6.73
598,600330,天通股份,10,8,11.44,9.87,29.90,23.91


In [82]:
# 机构持股详情

stock_institute_hold_detail_df = ak.stock_institute_hold_detail()
print(stock_institute_hold_detail_df)

None


In [ ]:
table_name_stock_institute_hold_detail_sina = 'stock_institute_hold_detail_sina'

stock_institute_hold_detail_df.to_sql(table_name_stock_institute_hold_detail_sina,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后

# 添加索引
# curs2.execute(
#     """
#     CREATE  INDEX stock_info_a_code_name_code  ON  stock_info_a_code_name  (code); 
#     """
# )
# conn2.commit()

In [ ]:
df_stock_institute_hold_detail_sina_2= pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name_stock_institute_hold_detail_sina}
  """,
   # params = (date,),
  con = engine2,
)

df_stock_institute_hold_detail_sina_2[:-5]

 ## 机构推荐股票
 "stock_institute_recommend"  # 机构推荐
 "stock_institute_recommend_detail"  # 股票评级记录

 ## A 股龙虎榜

 "stock_sina_lhb_detail_daily"  # 龙虎榜-每日详情

 "stock_sina_lhb_ggtj"  # 龙虎榜-个股上榜统计

 "stock_sina_lhb_yytj"  # 龙虎榜-营业上榜统计

 "stock_sina_lhb_jgzz"  # 龙虎榜-机构席位追踪
 
 "stock_sina_lhb_jgmx"  # 龙虎榜-机构席位成交明细